<a href="https://colab.research.google.com/github/won-seop/CAPSTONE-DESIGN/blob/main/chatgptAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
import openai
openai.api_key = ("") # -> gpt key

In [ ]:
df = pd.read_csv('drive/MyDrive/mydata.csv')
df = df[~df['keyword'].isnull()]
df

,title_data,keyword
0,나홀로 아이폰 사러 오사카,"나홀, 아이폰, 오사카"
1,김광석님이라니 대한민국 과학기술대전의 놀라운 현장 걱정하지 마세요 미래는 준비돼 있습니다,"대한민국, 걱정, 미래, 김광석님이라니"
2,노션 사용기 업무용 단기계획용 이력서용 노션 만드는 법,"노션, 이력서용, 사용기, 노션"
3,갤럭시 어떻게 나올까 정말 네이밍이 바뀔까 들려오는 소식들 정리했습니다,"네이밍, 정리, 갤럭시, 소식들"
4,갤럭시버즈라이브 에어팟프로 비교 들어갈게 최신 무선이어폰 사운드 통화연결 노캔 비교,"비교, 에어팟프, 무선이어폰, 사운드"
...,...,...
42803,대폭발 부동산이 미쳤다,"대폭발, 부동산"
42804,앞으로도 전세는 절망적이다,"앞, 전세, 절망적"
42805,충격 불과 일주일만에 이렇게나 떨어지다니집주인들 날벼락,"일주일, 떨어지다니집주인들, 날벼락, 충격"
42806,하반기 부동산 전망,"하반기, 부동산, 전망"


# **Completion 모델인 Davinci-002 사용**

In [ ]:
messages = []

for i, row in tqdm(df.iterrows()):
  try:
    message = {
        "prompt": df.iloc[i]["keyword"],
        "completion": df.iloc[i]["title_data"]
    }
    messages.append(message)
  except:
    break

In [ ]:
messages[:5]

In [ ]:
import json

with open("message.jsonl", "w", encoding="utf-8") as f:
  for entry in messages:
    json.dump(entry, f, ensure_ascii=False)
    f.write('\n')

In [ ]:
openai.File.create(
    file=open("message.jsonl", "rb"),
    purpose='fine-tune'
)

In [ ]:
id = "file-sgvm37RhC5E337eTaIbBsKhC"

openai.FineTuningJob.create(
    training_file=id,
    model="davinci-002"
)

In [ ]:
model_id = "ft:davinci-002:personal::7xsBgO2c"

user_input = "우울, 힐링, asmr" # keyword -> 제목 5개 뽑아줘

response = openai.Completion.create(
    model=model_id,
    prompt=user_input
)

In [ ]:
print(response['choices'][0]['text'])

# ***GPT-3.5 Turbo***
**Fine Tuning**

In [ ]:
messages = []

for i, row in tqdm(df.iterrows()):
  try:
    message = {
        "messages": [
            {"role": "system", "content": "ChatTitle is a Youtube title generator using keyword."},
            {"role": "user", "content": df.iloc[i]["keyword"]},
            {"role": "assistant", "content": df.iloc[i]["title_data"]}
        ]}

    messages.append(message)
  except:
    break

messages[-5:]

42730it [00:06, 6264.18it/s]


[{'messages': [{'role': 'system',
    'content': 'ChatTitle is a Youtube title generator using keyword.'},
   {'role': 'user', 'content': '대폭발, 부동산'},
   {'role': 'assistant', 'content': '대폭발 부동산이 미쳤다'}]},
 {'messages': [{'role': 'system',
    'content': 'ChatTitle is a Youtube title generator using keyword.'},
   {'role': 'user', 'content': '앞, 전세, 절망적'},
   {'role': 'assistant', 'content': '앞으로도 전세는 절망적이다'}]},
 {'messages': [{'role': 'system',
    'content': 'ChatTitle is a Youtube title generator using keyword.'},
   {'role': 'user', 'content': '일주일, 떨어지다니집주인들, 날벼락, 충격'},
   {'role': 'assistant', 'content': '충격 불과 일주일만에 이렇게나 떨어지다니집주인들 날벼락'}]},
 {'messages': [{'role': 'system',
    'content': 'ChatTitle is a Youtube title generator using keyword.'},
   {'role': 'user', 'content': '하반기, 부동산, 전망'},
   {'role': 'assistant', 'content': '하반기 부동산 전망'}]},
 {'messages': [{'role': 'system',
    'content': 'ChatTitle is a Youtube title generator using keyword.'},
   {'role': 'user', 'content':

In [ ]:
import json

with open("message.jsonl", "w", encoding="utf-8") as f:
  for entry in messages:
    json.dump(entry, f, ensure_ascii=False)
    f.write('\n')

In [ ]:
openai.File.create(
    file=open("message.jsonl", "rb"),
    purpose='fine-tune'
)

<File file id=file-B4L6e3BmKEIysRFD5QPQsKoj at 0x7dcdbc09d350> JSON: {
  "object": "file",
  "id": "file-B4L6e3BmKEIysRFD5QPQsKoj",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 12398253,
  "created_at": 1696322179,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
id = "file-B4L6e3BmKEIysRFD5QPQsKoj"

openai.FineTuningJob.create(
    training_file=id,
    model="gpt-3.5-turbo"
)

<FineTuningJob fine_tuning.job id=ftjob-m2X7iuHbCWlyiRTYytxgqVEW at 0x7a03d75b6610> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-m2X7iuHbCWlyiRTYytxgqVEW",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698735923,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-eOqVyK8J5nFVuDc41DlWu6aO",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-B4L6e3BmKEIysRFD5QPQsKoj",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [ ]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x7a59ef342f20> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-WUnakgqaehm0WpbnV1SoTXIF",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1696322201,
      "finished_at": 1696357750,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::85eQp6HM",
      "organization_id": "org-eOqVyK8J5nFVuDc41DlWu6aO",
      "result_files": [
        "file-38eGtYisUfyPF5pvFHUJSmmP"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-B4L6e3BmKEIysRFD5QPQsKoj",
      "hyperparameters": {
        "n_epochs": 1
      },
      "trained_tokens": 3417306,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-zw8aakKTdkNR2TErm3zn7zZs",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1694769074,
      "finished_at": 1694772533,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7z02oz39",
      "organization_id": "org-eOqVyK8J5n

In [ ]:
model_id = "ft:gpt-3.5-turbo-0613:personal::85eQp6HM"

user_input = "운동, 닭가슴살 ,식단 " # keyword

completions = []
for i in range(3):
  completion = openai.ChatCompletion.create(
      model=model_id,
      messages=[
          {"role": "system", "content": "ChatTitle is a youtube title generator using keyword."},
          {"role": "user", "content": user_input}
      ],
      max_tokens=70
  )
  completions.append(completion.choices[0].message["content"])


In [ ]:
for completion in completions:
  print(completion)

집 운동 걷기 닭가슴살 꽉찬  다이어트 식단 
근손실 없이 다이어트 운동식 닭가슴살만 먹으면 어떤 일이 발생하는가
한 달이면 운동원에 시달릴 것 같은데 코드개발식단 닭가슴살 코 쉽게 뻥튀기 시키는 굶는 방법


# **프롬프트로 제목 생성(파인튜닝X)**

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [
        {"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages
    )

    return response.choices[0].message["content"]

In [ ]:
keyword = "운동, 식단, 닭가슴살"

prompt = f"""너는 지금 부터 유튜브 채널을 운영하는 콘텐츠 크리에이터야.

키워드는 {keyword} 야.

넌, 내가 만든 유튜브 동영상의 제목을 만들어야해.

네가 만들어야할 “유튜브 동영상의 제목”은 아래 항목을 참고해서 만들어줘.

[동영상제목 만들때 참고사항]
1. 유튜브 검색 노출에 유리하도록 제목에 키워드를 넣어줘
2. 제목 앞 부분에 주목을 끌수 있는 창의적이고 재미있는 단어로 시작해줘. 이게 매우 중요해!
3. 주목을 끌만한 이모지를 여러개 사용해줘
4. 제목의 전체적인 톤앤매너는 “설명적”이고 “유익한”느낌으로 만들어줘
5. 길이는 한글로 70단어가 넘지 않도록 만들어줘

내가 가장 적합한 것을 사용할 수 있도록 3개의 제목을 만들어줘."""

response = get_completion(prompt)
print(response)

1. 경이로운 닭가슴살 다이어트, 4주만에 탄탄한 몸으로 변신하자! 💪🥗
2. 운동과 식단의 조합, 신나게 다이어트! 닭가슴살 레시피 공개 🏋️‍♀️🍽
3. 노력없는 다이어트는 없다! 닭가슴살로 빠르고 효과적으로 살 빼는 법 알려줌 😎🍗
